In [1]:
%pip install scipy
%pip install numpy
%pip install pandas
%pip install networkx
%pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
import networkx as nx
from scipy import sparse
from scipy.sparse.linalg import norm
from sklearn.model_selection import train_test_split
from typing import Tuple, Mapping, Any

In [3]:
df = pd.read_csv("res\\rating.csv")

In [4]:
df['rating'].describe()

count    2.000026e+07
mean     3.525529e+00
std      1.051989e+00
min      5.000000e-01
25%      3.000000e+00
50%      3.500000e+00
75%      4.000000e+00
max      5.000000e+00
Name: rating, dtype: float64

In [5]:
df.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


In [6]:
def split_by_target_column_value(df: pd.DataFrame, test_size: float, target_column: str, target_value: Any,
                                 random_state: int = 42) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """
    Разделяет DataFrame на обучающий и тестовый наборы на основе определенного значения целевого столбца.
    Выдает в качестве тестового набора test_size от набора, в котором значение в target_column == target_value
    В тренировочном все остальное
    Параметры:
        df (pd.DataFrame): DataFrame для разделения.
        test_size: Пропорция данных, которые следует включить в тестовый набор.
        target_column (str): Название целевого столбца для разделения.
        target_value (Any): Значение целевого столбца, используемое для разделения.
        random_state (int): Зерно для генератора случайных чисел.

    Возвращает:
        Tuple[pd.DataFrame, pd.Series, pd.DataFrame, pd.Series]: Данные для обучения; тестовые данные для целевой колонки, которые не попали в data,
        тренировочные данные для целевой колоники, которые попали в data
    """
    grouped = df.groupby(by=target_column)
    train_df, test_df = train_test_split(grouped.get_group(target_value), test_size=test_size,
                                         random_state=random_state)
    data = pd.concat([group for ind, group in grouped if ind != target_value], ignore_index=True)
    data = pd.concat([train_df, data], ignore_index=True)
    return data, test_df, train_df


In [7]:
data, test1, train1 = split_by_target_column_value(df, test_size=0.5, target_column='userId', target_value=1)

In [8]:
data.head()

,userId,movieId,rating,timestamp
0,1,2288,4.0,2004-09-10 03:14:37
1,1,2648,3.5,2004-09-10 03:13:37
2,1,1222,3.5,2005-04-02 23:30:37
3,1,3037,3.5,2005-04-02 23:46:03
4,1,2683,3.5,2004-09-10 03:07:30


In [9]:
test1.head()

,userId,movieId,rating,timestamp
156,1,7045,3.5,2005-04-02 23:50:34
145,1,6242,3.5,2004-09-10 03:13:24
101,1,3081,4.0,2004-09-10 03:15:43
127,1,4911,4.0,2005-04-02 23:52:41
141,1,5898,3.5,2005-04-02 23:53:22


In [10]:
train1.head()

,userId,movieId,rating,timestamp
79,1,2288,4.0,2004-09-10 03:14:37
84,1,2648,3.5,2004-09-10 03:13:37
39,1,1222,3.5,2005-04-02 23:30:37
100,1,3037,3.5,2005-04-02 23:46:03
86,1,2683,3.5,2004-09-10 03:07:30


In [3]:
from abc import ABC, abstractmethod
from typing import Mapping, Tuple, Any, Literal
MappingIntToObject = Mapping[int, Tuple[Any, Literal['source', 'target']]]
from sklearn.preprocessing import normalize
        
# https://github.com/danieljunhee/Tutorial-on-Personalized-PageRank/blob/master/Personalized_PageRank_Tutorial.ipynb
class PageRank():
    
    def __init__(self, data:pd.DataFrame, source: str, target:str, edge_attr:str,
                 personalize_by: Any, a: float = 0.8, b: float = 0.15, c: float=0.05,
                 max_iter: int = 100) -> None:
        # Decide values of a,b,c. For this tutorial, let's use a = 0.8, b = 0.15, c = 0.05.
        self.a = a
        self.b = b
        self.c = c
        self.max_iter = max_iter
        
        (self.__transition_matrix, self.__user_person_vec,
         self.__N_to_obj) = self.__create_transition_matrix_from_pd(data, source,
                                                             target, edge_attr, personalize_by)
         
        
    def __create_transition_matrix_from_pd(self, data: pd.DataFrame,
               source: str, target: str, edge_attr: str,
               personalization: Any) -> Tuple[sparse.csr_matrix, MappingIntToObject]:
    
        obj_source_to_N = {}
        obj_target_to_N = {}
        N_to_obj = {}
        
        source_uniq = set(data[source])
        for index, obj in zip(range(len(source_uniq)), source_uniq):
            obj_source_to_N[obj] = index
            N_to_obj[index] = (obj, 'source')
            
        
        target_uniq = set(data[target])
        for index, obj in zip(range(len(source_uniq), len(source_uniq) + len(target_uniq)), target_uniq):
            obj_target_to_N[obj] = index
            N_to_obj[index] = (obj, 'target')
        
        count_top = len(N_to_obj)
        transition_matrix = sparse.lil_matrix((count_top, count_top))
        
        # Функция для отображения значений согласно словарю
        def map_source_to_N(value):
            return obj_source_to_N[value]

        def map_target_to_N(value):
            return obj_target_to_N[value]

        # Создаем векторизованную версию функций
        vectorized_source_to_N = np.vectorize(map_source_to_N)
        vectorized_target_to_N = np.vectorize(map_target_to_N)

        # Применяем векторизованные функции к массивам
        start_nodes = vectorized_source_to_N(data[source].to_numpy())  # Начальные вершины ребер
        end_nodes = vectorized_target_to_N(data[target].to_numpy())    # Конечные вершины ребер
        weights = np.array(data[edge_attr])
        
        # Заполнение матрицы значениями из массивов
        transition_matrix[start_nodes, end_nodes] = weights

        # Если граф неориентированный, добавьте обратные ребра
        transition_matrix[end_nodes, start_nodes] = weights
        
        # Получаем суммы элементов каждой строки в виде numpy массива
        
        transition_matrix_csr = sparse.csr_matrix(transition_matrix)
        del transition_matrix
        
        row_sums = transition_matrix_csr.sum(axis=1)
        transition_matrix_csr /= row_sums
        
        user_person_vec = np.zeros(transition_matrix_csr.shape[1])
        #user_person_vec[obj_source_to_N[personalization]] = 1
        pers_data = data.loc[data[source] == personalization]
        for ind, row in pers_data.iterrows():
            user_person_vec[obj_target_to_N[row[target]]] = row[edge_attr]
        user_person_vec /= user_person_vec.sum()

        # transition_matrix_csr.getrow(obj_source_to_N[personalization]).transpose()
        return (transition_matrix_csr,
                sparse.csr_matrix(user_person_vec).transpose(), N_to_obj)
    
    def get_pagerank(self) -> pd.DataFrame:
        
        pagerank = self.__get_pagerank(self.__transition_matrix,
                                       self.__user_person_vec, self.max_iter)
         
         
        target_column = []
        pagerank_column = []
        for ind, el_pagerank in enumerate(pagerank):
            if self.__N_to_obj[ind][1] == 'target':
                target_column.append(self.__N_to_obj[ind][0])
                pagerank_column.append(el_pagerank)

        return pd.DataFrame({'target': target_column , 'pagerank': pagerank_column})
    
    def __get_pagerank(self, transition_matrix: sparse.csr_matrix,
                       user_person_vec: sparse.csr_matrix, max_iter: int = 200) -> np.ndarray:
        
        # Initialize PageRank vector: height x 1 vector of all 1/height 's
        # We also represent the PageRank vector as a sparse csr matrix so that every arithmetic operation result
        # is stored as a sparse csr matrix throughout the algorithm... again dealing with sparsity issue & memory issue
        
        height = transition_matrix.shape[0]
        user_pagerank_vec = sparse.csr_matrix(np.ones((height,1)) / height)
        sup_vec = self.c*sparse.csr_matrix((1/height)*np.ones((height,1)))

        # Update the PageRank vector until convergence! Our convergence criterion is to see whether
        # the magnitude of the difference of the PageRank vector after an update is 0
        converged = False
        count_iter = 0
        while (not converged):
            count_iter += 1
            # (deep) copy the current PageRank vector to compare with the updated vector
            old_user_pagerank_vec = user_pagerank_vec.copy()
            
            # Update!
            user_pagerank_vec = self.a * transition_matrix.dot(user_pagerank_vec) \
                                + self.b* user_person_vec.multiply(user_pagerank_vec) \
                                + sup_vec 
            user_pagerank_vec /= user_pagerank_vec.sum()
            
            # Compare with the PageRank vector before the update. Check the magnitude of the difference of the vector
            print("magnitude of difference vector:", norm(user_pagerank_vec-old_user_pagerank_vec))
            
            if norm(user_pagerank_vec-old_user_pagerank_vec) == 0 or count_iter == max_iter:
                converged = True
        return np.concatenate(user_pagerank_vec.toarray())
    
  
data_test = pd.DataFrame({
    'source': ['A', 'B', 'B', 'B'],
    'target': ['X', 'X', 'Z', 'C'],
    'edge_attr': [4, 4, 5, 3]
})  
    
pg = PageRank(data_test, 'source', 'target', 'edge_attr', 'A')
pg.get_pagerank()

KeyboardInterrupt: 

In [22]:
pg = PageRank(data, 'userId', 'movieId', 'rating', 1)

In [23]:
rank = pg.get_pagerank()

magnitude of difference vector: 1.067980329920182e-06
magnitude of difference vector: 1.9088230565322686e-07
magnitude of difference vector: 3.44072991278196e-08
magnitude of difference vector: 6.22529761805757e-09
magnitude of difference vector: 1.2319300673288039e-09
magnitude of difference vector: 5.171527673426363e-10
magnitude of difference vector: 4.277991516080747e-10
magnitude of difference vector: 4.15398952028087e-10
magnitude of difference vector: 3.788429810452005e-10
magnitude of difference vector: 3.6408704653875146e-10
magnitude of difference vector: 3.373324382289698e-10
magnitude of difference vector: 3.2064354181337997e-10
magnitude of difference vector: 2.9962436666274563e-10
magnitude of difference vector: 2.832306739197211e-10
magnitude of difference vector: 2.6573432001678353e-10
magnitude of difference vector: 2.5056912537882016e-10
magnitude of difference vector: 2.3551347002390603e-10
magnitude of difference vector: 2.2183354653862777e-10
magnitude of differenc

In [24]:
print(rank.sort_values(by='pagerank', ascending=True).head())


       target  pagerank
23657  112909  0.000006
23656  112907  0.000006
15519   78873  0.000006
24499  116704  0.000006
21197  103303  0.000006


In [25]:
rank.value_counts()

target  pagerank
1       0.000006    1
89386   0.000006    1
89753   0.000006    1
89747   0.000006    1
89745   0.000006    1
                   ..
26546   0.000006    1
26544   0.000006    1
26542   0.000006    1
26540   0.000006    1
131262  0.000006    1
Name: count, Length: 26744, dtype: int64

In [16]:
rank['pagerank'].sum()

0.16183844074004258